MSSV: 21120046.  
Họ tên: Lê Phan Minh Đạt

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from skimage.feature import hog
import numpy as np
import cv2
import time

In [2]:
# Hàm rút trích đặc trưng HOG từ hình ảnh
def extract_hog_features(images):
    hog_features = []
    hog = cv2.HOGDescriptor((28,28), (14,14), (7,7), (14,14), 9)  # Định nghĩa HOG descriptor
    for image in images:
        fd = hog.compute(image.numpy())  # Tính toán HOG
        hog_features.append(fd)
    return np.array(hog_features)

In [3]:
# Hàm rút trích đặc trưng Histogram từ hình ảnh
def extract_histogram_features(images):
    histogram_features = []
    for image in images:
        # Chuyển đổi hình ảnh sang định dạng dữ liệu phù hợp cho OpenCV
        image_np = image.numpy().astype(np.uint8)
        # Tính toán histogram
        hist = cv2.calcHist([image_np], [0], None, [256], [0, 256])
        # Chuẩn hóa histogram
        hist = hist.flatten() / np.sum(hist)
        histogram_features.append(hist)
    return np.array(histogram_features)

In [4]:
# Tải dữ liệu từ MNIST và chuẩn bị DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
])
mnist_train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
fashion_mnist_train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
fashion_mnist_test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:00<00:00, 33171404.26it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 1142121.53it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5017362.85it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 15489438.30it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [5]:
def run_model_one_hidden_size(hidden_size, train_data, test_data):

  # Hyperparameters
  input_size = 28 * 28 # Kích thước ảnh MNIST và Fashion MNIST
  num_classes = 10 # MNIST và Fashion MNIST có 10 lớp
  num_epochs = 5
  batch_size = 100
  learning_rate = 0.001
  # Device configuration
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=False)

  # Fully connected neural network with one hidden layer
  class NeuralNet(nn.Module):
      def __init__(self, input_size, hidden_size, num_classes):
          super(NeuralNet, self).__init__()
          self.fc1 = nn.Linear(input_size, hidden_size)
          self.relu = nn.ReLU()
          self.fc2 = nn.Linear(hidden_size, num_classes)

      def forward(self, x):
          out = self.fc1(x)
          out = self.relu(out)
          out = self.fc2(out)
          return out

  # Initialize the model
  model = NeuralNet(input_size, hidden_size, num_classes).to(device)

  # Loss and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Training loop
  def train_model(loader):
      model.train()
      total_step = len(loader)
      for epoch in range(num_epochs):
          for i, (images, labels) in enumerate(loader):
              # Move tensors to the configured device
              images = images.reshape(-1, 28*28).to(device)
              labels = labels.to(device)

              # Forward pass
              outputs = model(images)
              loss = criterion(outputs, labels)

              # Backward and optimize
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

              if (i+1) % 100 == 0:
                  print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

  # Validation function
  def validate_model(loader):
      model.eval()
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in loader:
              images = images.reshape(-1, 28*28).to(device)
              labels = labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

          print('Validate Accuracy: {} %'.format(100 * correct / total))

  # Test function
  def test_model(loader):
      model.eval()
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in loader:
              images = images.reshape(-1, 28*28).to(device)
              labels = labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

          print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

  start_time = time.time()
  print("Training on dataset...")
  train_model(train_loader)

  print("\nTest accuracy on dataset:")
  test_model(test_loader)
  end_time = time.time()
  execution_time = end_time - start_time
  print("Thời gian chạy:", execution_time, "giây")

In [6]:
def run_model_two_hidden_size(hidden_size1, hidden_size2, train_data, test_data):

    # Hyperparameters
    input_size = 28 * 28  # Kích thước ảnh MNIST và Fashion MNIST
    num_classes = 10  # MNIST và Fashion MNIST có 10 lớp
    num_epochs = 5
    batch_size = 100
    learning_rate = 0.001
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

    # Fully connected neural network with two hidden layers
    class NeuralNet(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
            super(NeuralNet, self).__init__()
            self.fc1 = nn.Linear(input_size, hidden_size1)
            self.relu1 = nn.ReLU()
            self.fc2 = nn.Linear(hidden_size1, hidden_size2)
            self.relu2 = nn.ReLU()
            self.fc3 = nn.Linear(hidden_size2, num_classes)

        def forward(self, x):
            out = self.fc1(x)
            out = self.relu1(out)
            out = self.fc2(out)
            out = self.relu2(out)
            out = self.fc3(out)
            return out

    # Initialize the model
    model = NeuralNet(input_size, hidden_size1, hidden_size2, num_classes).to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    def train_model(loader):
        model.train()
        total_step = len(loader)
        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(loader):
                # Move tensors to the configured device
                images = images.reshape(-1, 28 * 28).to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i + 1) % 100 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                          .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

    # Validation function
    def validate_model(loader):
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in loader:
                images = images.reshape(-1, 28 * 28).to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print('Validate Accuracy: {} %'.format(100 * correct / total))

    # Test function
    def test_model(loader):
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in loader:
                images = images.reshape(-1, 28 * 28).to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

    start_time = time.time()
    print("Training on dataset...")
    train_model(train_loader)

    print("\nTest accuracy on dataset:")
    test_model(test_loader)
    end_time = time.time()
    execution_time = end_time - start_time
    print("Thời gian chạy:", execution_time, "giây")

# Example usage:
# run_model(hidden_size1=256, hidden_size2=128, train_data=train_dataset, test_data=test_dataset)

In [7]:
def run_model_feature_one_hidden_size(hidden_size, train_data, test_data, HOG):
  if HOG:
    # Rút trích đặc trưng HOG từ tập huấn luyện và tập kiểm tra
    X_train_hog = extract_hog_features(train_data.data)
    X_test_hog = extract_hog_features(test_data.data)
  else:
    # Rút trích đặc trưng Histogram từ tập huấn luyện và tập kiểm tra
    X_train_hog = extract_histogram_features(train_data.data)
    X_test_hog = extract_histogram_features(test_data.data)
  # Chuyển đổi dữ liệu thành tensor và định dạng lại shape
  X_train_hog = torch.FloatTensor(X_train_hog)
  X_test_hog = torch.FloatTensor(X_test_hog)

  # Chuyển đổi nhãn sang tensor
  y_train = train_data.targets
  y_test = test_data.targets

  # Xây dựng DataLoader
  train_dataset = TensorDataset(X_train_hog, y_train)
  test_dataset = TensorDataset(X_test_hog, y_test)

  train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

  # Xây dựng mô hình mạng nơ-ron
  class NeuralNet(nn.Module):
      def __init__(self, input_size, hidden_size, num_classes):
          super(NeuralNet, self).__init__()
          self.fc1 = nn.Linear(input_size, hidden_size)
          self.relu = nn.ReLU()
          self.fc2 = nn.Linear(hidden_size, num_classes)

      def forward(self, x):
          out = self.fc1(x)
          out = self.relu(out)
          out = self.fc2(out)
          return out

  # Định nghĩa các siêu tham số
  input_size = X_train_hog.shape[1]
  num_classes = 10
  learning_rate = 0.001
  num_epochs = 5

  # Khởi tạo mô hình
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = NeuralNet(input_size, hidden_size, num_classes)

  # Hàm loss và optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  # Huấn luyện mô hình
  def train_model(loader):
        model.train()
        total_step = len(loader)
        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(loader):
                # Move tensors to the configured device
                labels = labels.to(device)
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i+1) % 100 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                          .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
  # Test function
  def test_model(loader):
      model.eval()
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in loader:
              labels = labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

          print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

  start_time = time.time()
  print("Training on dataset...")
  train_model(train_loader)

  print("\nTest accuracy on dataset:")
  test_model(test_loader)
  end_time = time.time()
  execution_time = end_time - start_time
  print("Thời gian chạy:", execution_time, "giây")

In [8]:
def run_model_feature_two_hidden_size(hidden_size1, hidden_size2, train_data, test_data, HOG):
  if HOG:
    # Rút trích đặc trưng HOG từ tập huấn luyện và tập kiểm tra
    X_train_hog = extract_hog_features(train_data.data)
    X_test_hog = extract_hog_features(test_data.data)
  else:
    # Rút trích đặc trưng Histogram từ tập huấn luyện và tập kiểm tra
    X_train_hog = extract_histogram_features(train_data.data)
    X_test_hog = extract_histogram_features(test_data.data)
  # Chuyển đổi dữ liệu thành tensor và định dạng lại shape
  X_train_hog = torch.FloatTensor(X_train_hog)
  X_test_hog = torch.FloatTensor(X_test_hog)

  # Chuyển đổi nhãn sang tensor
  y_train = train_data.targets
  y_test = test_data.targets

  # Xây dựng DataLoader
  train_dataset = TensorDataset(X_train_hog, y_train)
  test_dataset = TensorDataset(X_test_hog, y_test)

  train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

  # Fully connected neural network with two hidden layers
  class NeuralNet(nn.Module):
      def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
          super(NeuralNet, self).__init__()
          self.fc1 = nn.Linear(input_size, hidden_size1)
          self.relu1 = nn.ReLU()
          self.fc2 = nn.Linear(hidden_size1, hidden_size2)
          self.relu2 = nn.ReLU()
          self.fc3 = nn.Linear(hidden_size2, num_classes)

      def forward(self, x):
          out = self.fc1(x)
          out = self.relu1(out)
          out = self.fc2(out)
          out = self.relu2(out)
          out = self.fc3(out)
          return out

  input_size = X_train_hog.shape[1]
  num_classes = 10
  learning_rate = 0.001
  num_epochs = 5

  # Khởi tạo mô hình
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = NeuralNet(input_size, hidden_size1, hidden_size2, num_classes).to(device)

  # Hàm loss và optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  # Huấn luyện mô hình
  def train_model(loader):
        model.train()
        total_step = len(loader)
        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(loader):
                # Move tensors to the configured device
                labels = labels.to(device)
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                if (i+1) % 100 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                          .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
  # Test function
  def test_model(loader):
      model.eval()
      with torch.no_grad():
          correct = 0
          total = 0
          for images, labels in loader:
              labels = labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

          print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

  start_time = time.time()
  print("Training on dataset...")
  train_model(train_loader)

  print("\nTest accuracy on dataset:")
  test_model(test_loader)
  end_time = time.time()
  execution_time = end_time - start_time
  print("Thời gian chạy:", execution_time, "giây")

In [ ]:
run = run_model_one_hidden_size(100, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5044
Epoch [1/5], Step [200/600], Loss: 0.3139
Epoch [1/5], Step [300/600], Loss: 0.3070
Epoch [1/5], Step [400/600], Loss: 0.2952
Epoch [1/5], Step [500/600], Loss: 0.2600
Epoch [1/5], Step [600/600], Loss: 0.1566
Epoch [2/5], Step [100/600], Loss: 0.0920
Epoch [2/5], Step [200/600], Loss: 0.1961
Epoch [2/5], Step [300/600], Loss: 0.1135
Epoch [2/5], Step [400/600], Loss: 0.1541
Epoch [2/5], Step [500/600], Loss: 0.1411
Epoch [2/5], Step [600/600], Loss: 0.1895
Epoch [3/5], Step [100/600], Loss: 0.1393
Epoch [3/5], Step [200/600], Loss: 0.1299
Epoch [3/5], Step [300/600], Loss: 0.1142
Epoch [3/5], Step [400/600], Loss: 0.2028
Epoch [3/5], Step [500/600], Loss: 0.0687
Epoch [3/5], Step [600/600], Loss: 0.1385
Epoch [4/5], Step [100/600], Loss: 0.1489
Epoch [4/5], Step [200/600], Loss: 0.0920
Epoch [4/5], Step [300/600], Loss: 0.0634
Epoch [4/5], Step [400/600], Loss: 0.1948
Epoch [4/5], Step [500/600], Loss: 0.1377
Epoch [4/5]

In [ ]:
run = run_model_one_hidden_size(500, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3604
Epoch [1/5], Step [200/600], Loss: 0.2134
Epoch [1/5], Step [300/600], Loss: 0.3435
Epoch [1/5], Step [400/600], Loss: 0.1137
Epoch [1/5], Step [500/600], Loss: 0.1384
Epoch [1/5], Step [600/600], Loss: 0.2065
Epoch [2/5], Step [100/600], Loss: 0.1184
Epoch [2/5], Step [200/600], Loss: 0.0688
Epoch [2/5], Step [300/600], Loss: 0.1364
Epoch [2/5], Step [400/600], Loss: 0.1044
Epoch [2/5], Step [500/600], Loss: 0.1133
Epoch [2/5], Step [600/600], Loss: 0.1428
Epoch [3/5], Step [100/600], Loss: 0.1154
Epoch [3/5], Step [200/600], Loss: 0.0579
Epoch [3/5], Step [300/600], Loss: 0.0348
Epoch [3/5], Step [400/600], Loss: 0.0448
Epoch [3/5], Step [500/600], Loss: 0.0373
Epoch [3/5], Step [600/600], Loss: 0.1426
Epoch [4/5], Step [100/600], Loss: 0.1511
Epoch [4/5], Step [200/600], Loss: 0.0401
Epoch [4/5], Step [300/600], Loss: 0.1486
Epoch [4/5], Step [400/600], Loss: 0.0461
Epoch [4/5], Step [500/600], Loss: 0.0600
Epoch [4/5]

In [ ]:
run = run_model_one_hidden_size(700, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3377
Epoch [1/5], Step [200/600], Loss: 0.1901
Epoch [1/5], Step [300/600], Loss: 0.1692
Epoch [1/5], Step [400/600], Loss: 0.1589
Epoch [1/5], Step [500/600], Loss: 0.1794
Epoch [1/5], Step [600/600], Loss: 0.1748
Epoch [2/5], Step [100/600], Loss: 0.1056
Epoch [2/5], Step [200/600], Loss: 0.0429
Epoch [2/5], Step [300/600], Loss: 0.1356
Epoch [2/5], Step [400/600], Loss: 0.0998
Epoch [2/5], Step [500/600], Loss: 0.1664
Epoch [2/5], Step [600/600], Loss: 0.0801
Epoch [3/5], Step [100/600], Loss: 0.0699
Epoch [3/5], Step [200/600], Loss: 0.0353
Epoch [3/5], Step [300/600], Loss: 0.0351
Epoch [3/5], Step [400/600], Loss: 0.1032
Epoch [3/5], Step [500/600], Loss: 0.1364
Epoch [3/5], Step [600/600], Loss: 0.0430
Epoch [4/5], Step [100/600], Loss: 0.0894
Epoch [4/5], Step [200/600], Loss: 0.0176
Epoch [4/5], Step [300/600], Loss: 0.0286
Epoch [4/5], Step [400/600], Loss: 0.0248
Epoch [4/5], Step [500/600], Loss: 0.0114
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(100, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 1.1900
Epoch [1/5], Step [200/600], Loss: 0.5812
Epoch [1/5], Step [300/600], Loss: 0.3499
Epoch [1/5], Step [400/600], Loss: 0.3866
Epoch [1/5], Step [500/600], Loss: 0.2548
Epoch [1/5], Step [600/600], Loss: 0.2363
Epoch [2/5], Step [100/600], Loss: 0.1409
Epoch [2/5], Step [200/600], Loss: 0.1297
Epoch [2/5], Step [300/600], Loss: 0.2539
Epoch [2/5], Step [400/600], Loss: 0.2011
Epoch [2/5], Step [500/600], Loss: 0.1554
Epoch [2/5], Step [600/600], Loss: 0.2130
Epoch [3/5], Step [100/600], Loss: 0.2614
Epoch [3/5], Step [200/600], Loss: 0.1272
Epoch [3/5], Step [300/600], Loss: 0.1762
Epoch [3/5], Step [400/600], Loss: 0.2397
Epoch [3/5], Step [500/600], Loss: 0.1462
Epoch [3/5], Step [600/600], Loss: 0.1050
Epoch [4/5], Step [100/600], Loss: 0.1482
Epoch [4/5], Step [200/600], Loss: 0.1461
Epoch [4/5], Step [300/600], Loss: 0.0635
Epoch [4/5], Step [400/600], Loss: 0.1814
Epoch [4/5], Step [500/600], Loss: 0.1561
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(500, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5671
Epoch [1/5], Step [200/600], Loss: 0.2607
Epoch [1/5], Step [300/600], Loss: 0.3369
Epoch [1/5], Step [400/600], Loss: 0.2431
Epoch [1/5], Step [500/600], Loss: 0.1895
Epoch [1/5], Step [600/600], Loss: 0.0972
Epoch [2/5], Step [100/600], Loss: 0.1733
Epoch [2/5], Step [200/600], Loss: 0.1199
Epoch [2/5], Step [300/600], Loss: 0.1477
Epoch [2/5], Step [400/600], Loss: 0.1821
Epoch [2/5], Step [500/600], Loss: 0.1136
Epoch [2/5], Step [600/600], Loss: 0.0826
Epoch [3/5], Step [100/600], Loss: 0.1181
Epoch [3/5], Step [200/600], Loss: 0.0640
Epoch [3/5], Step [300/600], Loss: 0.1672
Epoch [3/5], Step [400/600], Loss: 0.1534
Epoch [3/5], Step [500/600], Loss: 0.0498
Epoch [3/5], Step [600/600], Loss: 0.1050
Epoch [4/5], Step [100/600], Loss: 0.0978
Epoch [4/5], Step [200/600], Loss: 0.0559
Epoch [4/5], Step [300/600], Loss: 0.0859
Epoch [4/5], Step [400/600], Loss: 0.0538
Epoch [4/5], Step [500/600], Loss: 0.0931
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(700, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.4439
Epoch [1/5], Step [200/600], Loss: 0.2830
Epoch [1/5], Step [300/600], Loss: 0.3041
Epoch [1/5], Step [400/600], Loss: 0.2149
Epoch [1/5], Step [500/600], Loss: 0.1429
Epoch [1/5], Step [600/600], Loss: 0.0735
Epoch [2/5], Step [100/600], Loss: 0.2296
Epoch [2/5], Step [200/600], Loss: 0.1346
Epoch [2/5], Step [300/600], Loss: 0.1751
Epoch [2/5], Step [400/600], Loss: 0.1725
Epoch [2/5], Step [500/600], Loss: 0.1969
Epoch [2/5], Step [600/600], Loss: 0.0806
Epoch [3/5], Step [100/600], Loss: 0.0737
Epoch [3/5], Step [200/600], Loss: 0.1041
Epoch [3/5], Step [300/600], Loss: 0.2195
Epoch [3/5], Step [400/600], Loss: 0.1438
Epoch [3/5], Step [500/600], Loss: 0.0912
Epoch [3/5], Step [600/600], Loss: 0.0646
Epoch [4/5], Step [100/600], Loss: 0.1756
Epoch [4/5], Step [200/600], Loss: 0.0907
Epoch [4/5], Step [300/600], Loss: 0.0640
Epoch [4/5], Step [400/600], Loss: 0.1619
Epoch [4/5], Step [500/600], Loss: 0.1166
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(100, 100, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3250
Epoch [1/5], Step [200/600], Loss: 0.3076
Epoch [1/5], Step [300/600], Loss: 0.3537
Epoch [1/5], Step [400/600], Loss: 0.2255
Epoch [1/5], Step [500/600], Loss: 0.1904
Epoch [1/5], Step [600/600], Loss: 0.2738
Epoch [2/5], Step [100/600], Loss: 0.1784
Epoch [2/5], Step [200/600], Loss: 0.0472
Epoch [2/5], Step [300/600], Loss: 0.1288
Epoch [2/5], Step [400/600], Loss: 0.1335
Epoch [2/5], Step [500/600], Loss: 0.1468
Epoch [2/5], Step [600/600], Loss: 0.1152
Epoch [3/5], Step [100/600], Loss: 0.1078
Epoch [3/5], Step [200/600], Loss: 0.0578
Epoch [3/5], Step [300/600], Loss: 0.1154
Epoch [3/5], Step [400/600], Loss: 0.0943
Epoch [3/5], Step [500/600], Loss: 0.1170
Epoch [3/5], Step [600/600], Loss: 0.1402
Epoch [4/5], Step [100/600], Loss: 0.1145
Epoch [4/5], Step [200/600], Loss: 0.1375
Epoch [4/5], Step [300/600], Loss: 0.0278
Epoch [4/5], Step [400/600], Loss: 0.1016
Epoch [4/5], Step [500/600], Loss: 0.0964
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(100, 500, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3528
Epoch [1/5], Step [200/600], Loss: 0.2375
Epoch [1/5], Step [300/600], Loss: 0.2711
Epoch [1/5], Step [400/600], Loss: 0.1849
Epoch [1/5], Step [500/600], Loss: 0.3221
Epoch [1/5], Step [600/600], Loss: 0.1103
Epoch [2/5], Step [100/600], Loss: 0.1588
Epoch [2/5], Step [200/600], Loss: 0.0539
Epoch [2/5], Step [300/600], Loss: 0.1563
Epoch [2/5], Step [400/600], Loss: 0.0911
Epoch [2/5], Step [500/600], Loss: 0.0500
Epoch [2/5], Step [600/600], Loss: 0.1741
Epoch [3/5], Step [100/600], Loss: 0.1701
Epoch [3/5], Step [200/600], Loss: 0.0457
Epoch [3/5], Step [300/600], Loss: 0.1279
Epoch [3/5], Step [400/600], Loss: 0.0628
Epoch [3/5], Step [500/600], Loss: 0.0552
Epoch [3/5], Step [600/600], Loss: 0.0384
Epoch [4/5], Step [100/600], Loss: 0.0238
Epoch [4/5], Step [200/600], Loss: 0.0697
Epoch [4/5], Step [300/600], Loss: 0.0496
Epoch [4/5], Step [400/600], Loss: 0.0971
Epoch [4/5], Step [500/600], Loss: 0.0668
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(100, 700, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.2040
Epoch [1/5], Step [200/600], Loss: 0.2547
Epoch [1/5], Step [300/600], Loss: 0.2132
Epoch [1/5], Step [400/600], Loss: 0.1632
Epoch [1/5], Step [500/600], Loss: 0.2277
Epoch [1/5], Step [600/600], Loss: 0.1443
Epoch [2/5], Step [100/600], Loss: 0.1203
Epoch [2/5], Step [200/600], Loss: 0.0617
Epoch [2/5], Step [300/600], Loss: 0.1992
Epoch [2/5], Step [400/600], Loss: 0.2500
Epoch [2/5], Step [500/600], Loss: 0.0394
Epoch [2/5], Step [600/600], Loss: 0.0624
Epoch [3/5], Step [100/600], Loss: 0.0571
Epoch [3/5], Step [200/600], Loss: 0.0326
Epoch [3/5], Step [300/600], Loss: 0.1724
Epoch [3/5], Step [400/600], Loss: 0.0870
Epoch [3/5], Step [500/600], Loss: 0.1254
Epoch [3/5], Step [600/600], Loss: 0.0268
Epoch [4/5], Step [100/600], Loss: 0.0366
Epoch [4/5], Step [200/600], Loss: 0.0099
Epoch [4/5], Step [300/600], Loss: 0.1599
Epoch [4/5], Step [400/600], Loss: 0.0489
Epoch [4/5], Step [500/600], Loss: 0.0141
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(500, 100, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.4171
Epoch [1/5], Step [200/600], Loss: 0.3544
Epoch [1/5], Step [300/600], Loss: 0.1900
Epoch [1/5], Step [400/600], Loss: 0.1627
Epoch [1/5], Step [500/600], Loss: 0.2191
Epoch [1/5], Step [600/600], Loss: 0.2387
Epoch [2/5], Step [100/600], Loss: 0.1100
Epoch [2/5], Step [200/600], Loss: 0.1102
Epoch [2/5], Step [300/600], Loss: 0.0602
Epoch [2/5], Step [400/600], Loss: 0.1090
Epoch [2/5], Step [500/600], Loss: 0.0847
Epoch [2/5], Step [600/600], Loss: 0.1119
Epoch [3/5], Step [100/600], Loss: 0.0824
Epoch [3/5], Step [200/600], Loss: 0.0419
Epoch [3/5], Step [300/600], Loss: 0.0349
Epoch [3/5], Step [400/600], Loss: 0.0357
Epoch [3/5], Step [500/600], Loss: 0.0847
Epoch [3/5], Step [600/600], Loss: 0.0381
Epoch [4/5], Step [100/600], Loss: 0.0325
Epoch [4/5], Step [200/600], Loss: 0.0473
Epoch [4/5], Step [300/600], Loss: 0.0588
Epoch [4/5], Step [400/600], Loss: 0.0711
Epoch [4/5], Step [500/600], Loss: 0.0179
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(500, 500, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3031
Epoch [1/5], Step [200/600], Loss: 0.2185
Epoch [1/5], Step [300/600], Loss: 0.1564
Epoch [1/5], Step [400/600], Loss: 0.1297
Epoch [1/5], Step [500/600], Loss: 0.2996
Epoch [1/5], Step [600/600], Loss: 0.1031
Epoch [2/5], Step [100/600], Loss: 0.1160
Epoch [2/5], Step [200/600], Loss: 0.0940
Epoch [2/5], Step [300/600], Loss: 0.0423
Epoch [2/5], Step [400/600], Loss: 0.0210
Epoch [2/5], Step [500/600], Loss: 0.0727
Epoch [2/5], Step [600/600], Loss: 0.0340
Epoch [3/5], Step [100/600], Loss: 0.0735
Epoch [3/5], Step [200/600], Loss: 0.2083
Epoch [3/5], Step [300/600], Loss: 0.0185
Epoch [3/5], Step [400/600], Loss: 0.0636
Epoch [3/5], Step [500/600], Loss: 0.0482
Epoch [3/5], Step [600/600], Loss: 0.0267
Epoch [4/5], Step [100/600], Loss: 0.0291
Epoch [4/5], Step [200/600], Loss: 0.0119
Epoch [4/5], Step [300/600], Loss: 0.0045
Epoch [4/5], Step [400/600], Loss: 0.0608
Epoch [4/5], Step [500/600], Loss: 0.0044
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(500, 700, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.2982
Epoch [1/5], Step [200/600], Loss: 0.2347
Epoch [1/5], Step [300/600], Loss: 0.1210
Epoch [1/5], Step [400/600], Loss: 0.1802
Epoch [1/5], Step [500/600], Loss: 0.0843
Epoch [1/5], Step [600/600], Loss: 0.1280
Epoch [2/5], Step [100/600], Loss: 0.1627
Epoch [2/5], Step [200/600], Loss: 0.0353
Epoch [2/5], Step [300/600], Loss: 0.0808
Epoch [2/5], Step [400/600], Loss: 0.0471
Epoch [2/5], Step [500/600], Loss: 0.0722
Epoch [2/5], Step [600/600], Loss: 0.1462
Epoch [3/5], Step [100/600], Loss: 0.0304
Epoch [3/5], Step [200/600], Loss: 0.0229
Epoch [3/5], Step [300/600], Loss: 0.0722
Epoch [3/5], Step [400/600], Loss: 0.1305
Epoch [3/5], Step [500/600], Loss: 0.0976
Epoch [3/5], Step [600/600], Loss: 0.0889
Epoch [4/5], Step [100/600], Loss: 0.0376
Epoch [4/5], Step [200/600], Loss: 0.0065
Epoch [4/5], Step [300/600], Loss: 0.1022
Epoch [4/5], Step [400/600], Loss: 0.0403
Epoch [4/5], Step [500/600], Loss: 0.0426
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(700, 100, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3127
Epoch [1/5], Step [200/600], Loss: 0.3768
Epoch [1/5], Step [300/600], Loss: 0.2665
Epoch [1/5], Step [400/600], Loss: 0.2157
Epoch [1/5], Step [500/600], Loss: 0.1886
Epoch [1/5], Step [600/600], Loss: 0.0868
Epoch [2/5], Step [100/600], Loss: 0.1147
Epoch [2/5], Step [200/600], Loss: 0.1177
Epoch [2/5], Step [300/600], Loss: 0.0703
Epoch [2/5], Step [400/600], Loss: 0.0636
Epoch [2/5], Step [500/600], Loss: 0.1308
Epoch [2/5], Step [600/600], Loss: 0.1631
Epoch [3/5], Step [100/600], Loss: 0.0522
Epoch [3/5], Step [200/600], Loss: 0.1135
Epoch [3/5], Step [300/600], Loss: 0.0702
Epoch [3/5], Step [400/600], Loss: 0.0121
Epoch [3/5], Step [500/600], Loss: 0.0782
Epoch [3/5], Step [600/600], Loss: 0.1495
Epoch [4/5], Step [100/600], Loss: 0.0398
Epoch [4/5], Step [200/600], Loss: 0.0624
Epoch [4/5], Step [300/600], Loss: 0.0481
Epoch [4/5], Step [400/600], Loss: 0.0555
Epoch [4/5], Step [500/600], Loss: 0.0343
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(700, 500, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.4452
Epoch [1/5], Step [200/600], Loss: 0.1638
Epoch [1/5], Step [300/600], Loss: 0.1664
Epoch [1/5], Step [400/600], Loss: 0.1427
Epoch [1/5], Step [500/600], Loss: 0.1364
Epoch [1/5], Step [600/600], Loss: 0.2172
Epoch [2/5], Step [100/600], Loss: 0.0790
Epoch [2/5], Step [200/600], Loss: 0.0317
Epoch [2/5], Step [300/600], Loss: 0.0664
Epoch [2/5], Step [400/600], Loss: 0.0257
Epoch [2/5], Step [500/600], Loss: 0.0080
Epoch [2/5], Step [600/600], Loss: 0.0966
Epoch [3/5], Step [100/600], Loss: 0.0689
Epoch [3/5], Step [200/600], Loss: 0.0649
Epoch [3/5], Step [300/600], Loss: 0.0366
Epoch [3/5], Step [400/600], Loss: 0.0489
Epoch [3/5], Step [500/600], Loss: 0.1721
Epoch [3/5], Step [600/600], Loss: 0.0123
Epoch [4/5], Step [100/600], Loss: 0.0454
Epoch [4/5], Step [200/600], Loss: 0.0295
Epoch [4/5], Step [300/600], Loss: 0.0171
Epoch [4/5], Step [400/600], Loss: 0.0137
Epoch [4/5], Step [500/600], Loss: 0.0516
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(700, 700, mnist_train_data, mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.2305
Epoch [1/5], Step [200/600], Loss: 0.1025
Epoch [1/5], Step [300/600], Loss: 0.1634
Epoch [1/5], Step [400/600], Loss: 0.2112
Epoch [1/5], Step [500/600], Loss: 0.1886
Epoch [1/5], Step [600/600], Loss: 0.1068
Epoch [2/5], Step [100/600], Loss: 0.1129
Epoch [2/5], Step [200/600], Loss: 0.0651
Epoch [2/5], Step [300/600], Loss: 0.0490
Epoch [2/5], Step [400/600], Loss: 0.0839
Epoch [2/5], Step [500/600], Loss: 0.0100
Epoch [2/5], Step [600/600], Loss: 0.0343
Epoch [3/5], Step [100/600], Loss: 0.0664
Epoch [3/5], Step [200/600], Loss: 0.0734
Epoch [3/5], Step [300/600], Loss: 0.0720
Epoch [3/5], Step [400/600], Loss: 0.0810
Epoch [3/5], Step [500/600], Loss: 0.0536
Epoch [3/5], Step [600/600], Loss: 0.0871
Epoch [4/5], Step [100/600], Loss: 0.0638
Epoch [4/5], Step [200/600], Loss: 0.0075
Epoch [4/5], Step [300/600], Loss: 0.0231
Epoch [4/5], Step [400/600], Loss: 0.0965
Epoch [4/5], Step [500/600], Loss: 0.0623
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(100, 100, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.6670
Epoch [1/5], Step [200/600], Loss: 0.3385
Epoch [1/5], Step [300/600], Loss: 0.1766
Epoch [1/5], Step [400/600], Loss: 0.2819
Epoch [1/5], Step [500/600], Loss: 0.1705
Epoch [1/5], Step [600/600], Loss: 0.1644
Epoch [2/5], Step [100/600], Loss: 0.2005
Epoch [2/5], Step [200/600], Loss: 0.1544
Epoch [2/5], Step [300/600], Loss: 0.1885
Epoch [2/5], Step [400/600], Loss: 0.1686
Epoch [2/5], Step [500/600], Loss: 0.0823
Epoch [2/5], Step [600/600], Loss: 0.1032
Epoch [3/5], Step [100/600], Loss: 0.1156
Epoch [3/5], Step [200/600], Loss: 0.0738
Epoch [3/5], Step [300/600], Loss: 0.1882
Epoch [3/5], Step [400/600], Loss: 0.1560
Epoch [3/5], Step [500/600], Loss: 0.1412
Epoch [3/5], Step [600/600], Loss: 0.0973
Epoch [4/5], Step [100/600], Loss: 0.1006
Epoch [4/5], Step [200/600], Loss: 0.0419
Epoch [4/5], Step [300/600], Loss: 0.1303
Epoch [4/5], Step [400/600], Loss: 0.1013
Epoch [4/5], Step [500/600], Loss: 0.0951
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(100, 500, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3834
Epoch [1/5], Step [200/600], Loss: 0.2855
Epoch [1/5], Step [300/600], Loss: 0.2037
Epoch [1/5], Step [400/600], Loss: 0.2743
Epoch [1/5], Step [500/600], Loss: 0.1541
Epoch [1/5], Step [600/600], Loss: 0.2473
Epoch [2/5], Step [100/600], Loss: 0.1212
Epoch [2/5], Step [200/600], Loss: 0.0864
Epoch [2/5], Step [300/600], Loss: 0.0871
Epoch [2/5], Step [400/600], Loss: 0.1211
Epoch [2/5], Step [500/600], Loss: 0.0708
Epoch [2/5], Step [600/600], Loss: 0.0825
Epoch [3/5], Step [100/600], Loss: 0.1597
Epoch [3/5], Step [200/600], Loss: 0.0713
Epoch [3/5], Step [300/600], Loss: 0.1173
Epoch [3/5], Step [400/600], Loss: 0.1305
Epoch [3/5], Step [500/600], Loss: 0.1090
Epoch [3/5], Step [600/600], Loss: 0.2621
Epoch [4/5], Step [100/600], Loss: 0.2895
Epoch [4/5], Step [200/600], Loss: 0.0614
Epoch [4/5], Step [300/600], Loss: 0.0266
Epoch [4/5], Step [400/600], Loss: 0.1495
Epoch [4/5], Step [500/600], Loss: 0.0760
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(100, 700, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3383
Epoch [1/5], Step [200/600], Loss: 0.2399
Epoch [1/5], Step [300/600], Loss: 0.2211
Epoch [1/5], Step [400/600], Loss: 0.2221
Epoch [1/5], Step [500/600], Loss: 0.1243
Epoch [1/5], Step [600/600], Loss: 0.2158
Epoch [2/5], Step [100/600], Loss: 0.1157
Epoch [2/5], Step [200/600], Loss: 0.2299
Epoch [2/5], Step [300/600], Loss: 0.2553
Epoch [2/5], Step [400/600], Loss: 0.1146
Epoch [2/5], Step [500/600], Loss: 0.0878
Epoch [2/5], Step [600/600], Loss: 0.2167
Epoch [3/5], Step [100/600], Loss: 0.0942
Epoch [3/5], Step [200/600], Loss: 0.1818
Epoch [3/5], Step [300/600], Loss: 0.1524
Epoch [3/5], Step [400/600], Loss: 0.0891
Epoch [3/5], Step [500/600], Loss: 0.1172
Epoch [3/5], Step [600/600], Loss: 0.0813
Epoch [4/5], Step [100/600], Loss: 0.2134
Epoch [4/5], Step [200/600], Loss: 0.1064
Epoch [4/5], Step [300/600], Loss: 0.0699
Epoch [4/5], Step [400/600], Loss: 0.0358
Epoch [4/5], Step [500/600], Loss: 0.0825
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(500, 100, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3903
Epoch [1/5], Step [200/600], Loss: 0.2193
Epoch [1/5], Step [300/600], Loss: 0.3066
Epoch [1/5], Step [400/600], Loss: 0.1841
Epoch [1/5], Step [500/600], Loss: 0.1762
Epoch [1/5], Step [600/600], Loss: 0.1559
Epoch [2/5], Step [100/600], Loss: 0.0629
Epoch [2/5], Step [200/600], Loss: 0.1454
Epoch [2/5], Step [300/600], Loss: 0.1326
Epoch [2/5], Step [400/600], Loss: 0.1122
Epoch [2/5], Step [500/600], Loss: 0.0841
Epoch [2/5], Step [600/600], Loss: 0.1652
Epoch [3/5], Step [100/600], Loss: 0.0938
Epoch [3/5], Step [200/600], Loss: 0.2013
Epoch [3/5], Step [300/600], Loss: 0.1461
Epoch [3/5], Step [400/600], Loss: 0.1470
Epoch [3/5], Step [500/600], Loss: 0.1622
Epoch [3/5], Step [600/600], Loss: 0.0870
Epoch [4/5], Step [100/600], Loss: 0.0601
Epoch [4/5], Step [200/600], Loss: 0.0682
Epoch [4/5], Step [300/600], Loss: 0.1136
Epoch [4/5], Step [400/600], Loss: 0.0777
Epoch [4/5], Step [500/600], Loss: 0.1855
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(500, 500, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.4181
Epoch [1/5], Step [200/600], Loss: 0.3329
Epoch [1/5], Step [300/600], Loss: 0.1536
Epoch [1/5], Step [400/600], Loss: 0.2017
Epoch [1/5], Step [500/600], Loss: 0.1123
Epoch [1/5], Step [600/600], Loss: 0.0901
Epoch [2/5], Step [100/600], Loss: 0.0878
Epoch [2/5], Step [200/600], Loss: 0.0949
Epoch [2/5], Step [300/600], Loss: 0.0868
Epoch [2/5], Step [400/600], Loss: 0.2347
Epoch [2/5], Step [500/600], Loss: 0.1149
Epoch [2/5], Step [600/600], Loss: 0.1946
Epoch [3/5], Step [100/600], Loss: 0.1233
Epoch [3/5], Step [200/600], Loss: 0.0910
Epoch [3/5], Step [300/600], Loss: 0.1593
Epoch [3/5], Step [400/600], Loss: 0.1044
Epoch [3/5], Step [500/600], Loss: 0.0308
Epoch [3/5], Step [600/600], Loss: 0.0650
Epoch [4/5], Step [100/600], Loss: 0.0871
Epoch [4/5], Step [200/600], Loss: 0.0480
Epoch [4/5], Step [300/600], Loss: 0.1291
Epoch [4/5], Step [400/600], Loss: 0.1146
Epoch [4/5], Step [500/600], Loss: 0.0679
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(500, 700, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.2527
Epoch [1/5], Step [200/600], Loss: 0.1272
Epoch [1/5], Step [300/600], Loss: 0.0479
Epoch [1/5], Step [400/600], Loss: 0.0471
Epoch [1/5], Step [500/600], Loss: 0.0655
Epoch [1/5], Step [600/600], Loss: 0.2112
Epoch [2/5], Step [100/600], Loss: 0.1842
Epoch [2/5], Step [200/600], Loss: 0.1578
Epoch [2/5], Step [300/600], Loss: 0.0778
Epoch [2/5], Step [400/600], Loss: 0.0699
Epoch [2/5], Step [500/600], Loss: 0.1045
Epoch [2/5], Step [600/600], Loss: 0.0710
Epoch [3/5], Step [100/600], Loss: 0.0762
Epoch [3/5], Step [200/600], Loss: 0.0642
Epoch [3/5], Step [300/600], Loss: 0.0991
Epoch [3/5], Step [400/600], Loss: 0.0259
Epoch [3/5], Step [500/600], Loss: 0.1311
Epoch [3/5], Step [600/600], Loss: 0.0504
Epoch [4/5], Step [100/600], Loss: 0.1456
Epoch [4/5], Step [200/600], Loss: 0.0953
Epoch [4/5], Step [300/600], Loss: 0.0383
Epoch [4/5], Step [400/600], Loss: 0.0921
Epoch [4/5], Step [500/600], Loss: 0.0693
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(700, 100, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.3552
Epoch [1/5], Step [200/600], Loss: 0.3601
Epoch [1/5], Step [300/600], Loss: 0.1448
Epoch [1/5], Step [400/600], Loss: 0.1976
Epoch [1/5], Step [500/600], Loss: 0.1213
Epoch [1/5], Step [600/600], Loss: 0.1603
Epoch [2/5], Step [100/600], Loss: 0.0834
Epoch [2/5], Step [200/600], Loss: 0.1256
Epoch [2/5], Step [300/600], Loss: 0.1105
Epoch [2/5], Step [400/600], Loss: 0.1450
Epoch [2/5], Step [500/600], Loss: 0.0576
Epoch [2/5], Step [600/600], Loss: 0.1833
Epoch [3/5], Step [100/600], Loss: 0.1233
Epoch [3/5], Step [200/600], Loss: 0.0666
Epoch [3/5], Step [300/600], Loss: 0.0921
Epoch [3/5], Step [400/600], Loss: 0.1779
Epoch [3/5], Step [500/600], Loss: 0.0834
Epoch [3/5], Step [600/600], Loss: 0.1963
Epoch [4/5], Step [100/600], Loss: 0.1312
Epoch [4/5], Step [200/600], Loss: 0.0729
Epoch [4/5], Step [300/600], Loss: 0.1746
Epoch [4/5], Step [400/600], Loss: 0.0957
Epoch [4/5], Step [500/600], Loss: 0.0636
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(700, 500, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.2548
Epoch [1/5], Step [200/600], Loss: 0.2629
Epoch [1/5], Step [300/600], Loss: 0.1050
Epoch [1/5], Step [400/600], Loss: 0.1465
Epoch [1/5], Step [500/600], Loss: 0.1564
Epoch [1/5], Step [600/600], Loss: 0.2218
Epoch [2/5], Step [100/600], Loss: 0.1819
Epoch [2/5], Step [200/600], Loss: 0.1881
Epoch [2/5], Step [300/600], Loss: 0.1160
Epoch [2/5], Step [400/600], Loss: 0.1093
Epoch [2/5], Step [500/600], Loss: 0.1282
Epoch [2/5], Step [600/600], Loss: 0.1482
Epoch [3/5], Step [100/600], Loss: 0.1124
Epoch [3/5], Step [200/600], Loss: 0.1873
Epoch [3/5], Step [300/600], Loss: 0.1196
Epoch [3/5], Step [400/600], Loss: 0.0901
Epoch [3/5], Step [500/600], Loss: 0.0472
Epoch [3/5], Step [600/600], Loss: 0.0694
Epoch [4/5], Step [100/600], Loss: 0.0874
Epoch [4/5], Step [200/600], Loss: 0.0611
Epoch [4/5], Step [300/600], Loss: 0.1291
Epoch [4/5], Step [400/600], Loss: 0.0729
Epoch [4/5], Step [500/600], Loss: 0.0714
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(700, 700, mnist_train_data, mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.2631
Epoch [1/5], Step [200/600], Loss: 0.2364
Epoch [1/5], Step [300/600], Loss: 0.1184
Epoch [1/5], Step [400/600], Loss: 0.1560
Epoch [1/5], Step [500/600], Loss: 0.1550
Epoch [1/5], Step [600/600], Loss: 0.0779
Epoch [2/5], Step [100/600], Loss: 0.1730
Epoch [2/5], Step [200/600], Loss: 0.1183
Epoch [2/5], Step [300/600], Loss: 0.0768
Epoch [2/5], Step [400/600], Loss: 0.1400
Epoch [2/5], Step [500/600], Loss: 0.1522
Epoch [2/5], Step [600/600], Loss: 0.1912
Epoch [3/5], Step [100/600], Loss: 0.0278
Epoch [3/5], Step [200/600], Loss: 0.0931
Epoch [3/5], Step [300/600], Loss: 0.0633
Epoch [3/5], Step [400/600], Loss: 0.1287
Epoch [3/5], Step [500/600], Loss: 0.0803
Epoch [3/5], Step [600/600], Loss: 0.0702
Epoch [4/5], Step [100/600], Loss: 0.2083
Epoch [4/5], Step [200/600], Loss: 0.1497
Epoch [4/5], Step [300/600], Loss: 0.0520
Epoch [4/5], Step [400/600], Loss: 0.0760
Epoch [4/5], Step [500/600], Loss: 0.0880
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(700, mnist_train_data, mnist_test_data, False)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 2.2675
Epoch [1/5], Step [200/600], Loss: 2.1960
Epoch [1/5], Step [300/600], Loss: 2.0343
Epoch [1/5], Step [400/600], Loss: 1.8840
Epoch [1/5], Step [500/600], Loss: 1.9406
Epoch [1/5], Step [600/600], Loss: 1.8919
Epoch [2/5], Step [100/600], Loss: 1.8768
Epoch [2/5], Step [200/600], Loss: 1.7653
Epoch [2/5], Step [300/600], Loss: 1.8608
Epoch [2/5], Step [400/600], Loss: 1.8205
Epoch [2/5], Step [500/600], Loss: 1.8788
Epoch [2/5], Step [600/600], Loss: 1.8978
Epoch [3/5], Step [100/600], Loss: 1.7848
Epoch [3/5], Step [200/600], Loss: 1.8017
Epoch [3/5], Step [300/600], Loss: 1.6871
Epoch [3/5], Step [400/600], Loss: 1.8914
Epoch [3/5], Step [500/600], Loss: 1.8514
Epoch [3/5], Step [600/600], Loss: 1.8358
Epoch [4/5], Step [100/600], Loss: 1.8686
Epoch [4/5], Step [200/600], Loss: 1.7106
Epoch [4/5], Step [300/600], Loss: 1.8110
Epoch [4/5], Step [400/600], Loss: 1.9075
Epoch [4/5], Step [500/600], Loss: 1.8084
Epoch [4/5]

In [9]:
run = run_model_feature_two_hidden_size(500, 500, mnist_train_data, mnist_test_data, False)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 2.1287
Epoch [1/5], Step [200/600], Loss: 1.8426
Epoch [1/5], Step [300/600], Loss: 1.8475
Epoch [1/5], Step [400/600], Loss: 1.8308
Epoch [1/5], Step [500/600], Loss: 1.9268
Epoch [1/5], Step [600/600], Loss: 1.7828
Epoch [2/5], Step [100/600], Loss: 1.8942
Epoch [2/5], Step [200/600], Loss: 1.7362
Epoch [2/5], Step [300/600], Loss: 1.9739
Epoch [2/5], Step [400/600], Loss: 1.8414
Epoch [2/5], Step [500/600], Loss: 1.7344
Epoch [2/5], Step [600/600], Loss: 1.7490
Epoch [3/5], Step [100/600], Loss: 1.9079
Epoch [3/5], Step [200/600], Loss: 1.6607
Epoch [3/5], Step [300/600], Loss: 1.8117
Epoch [3/5], Step [400/600], Loss: 1.6659
Epoch [3/5], Step [500/600], Loss: 1.8710
Epoch [3/5], Step [600/600], Loss: 1.9189
Epoch [4/5], Step [100/600], Loss: 1.8750
Epoch [4/5], Step [200/600], Loss: 1.5938
Epoch [4/5], Step [300/600], Loss: 1.8277
Epoch [4/5], Step [400/600], Loss: 1.6398
Epoch [4/5], Step [500/600], Loss: 1.7396
Epoch [4/5]

In [ ]:
run = run_model_one_hidden_size(100, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.6036
Epoch [1/5], Step [200/600], Loss: 0.6699
Epoch [1/5], Step [300/600], Loss: 0.5925
Epoch [1/5], Step [400/600], Loss: 0.4635
Epoch [1/5], Step [500/600], Loss: 0.3876
Epoch [1/5], Step [600/600], Loss: 0.3378
Epoch [2/5], Step [100/600], Loss: 0.4256
Epoch [2/5], Step [200/600], Loss: 0.3489
Epoch [2/5], Step [300/600], Loss: 0.4274
Epoch [2/5], Step [400/600], Loss: 0.2824
Epoch [2/5], Step [500/600], Loss: 0.3589
Epoch [2/5], Step [600/600], Loss: 0.4257
Epoch [3/5], Step [100/600], Loss: 0.4096
Epoch [3/5], Step [200/600], Loss: 0.3844
Epoch [3/5], Step [300/600], Loss: 0.2561
Epoch [3/5], Step [400/600], Loss: 0.4566
Epoch [3/5], Step [500/600], Loss: 0.4272
Epoch [3/5], Step [600/600], Loss: 0.4387
Epoch [4/5], Step [100/600], Loss: 0.3759
Epoch [4/5], Step [200/600], Loss: 0.3280
Epoch [4/5], Step [300/600], Loss: 0.3798
Epoch [4/5], Step [400/600], Loss: 0.2339
Epoch [4/5], Step [500/600], Loss: 0.3415
Epoch [4/5]

In [ ]:
run = run_model_one_hidden_size(500, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.4966
Epoch [1/5], Step [200/600], Loss: 0.3811
Epoch [1/5], Step [300/600], Loss: 0.6893
Epoch [1/5], Step [400/600], Loss: 0.4539
Epoch [1/5], Step [500/600], Loss: 0.3509
Epoch [1/5], Step [600/600], Loss: 0.4314
Epoch [2/5], Step [100/600], Loss: 0.5748
Epoch [2/5], Step [200/600], Loss: 0.4241
Epoch [2/5], Step [300/600], Loss: 0.3864
Epoch [2/5], Step [400/600], Loss: 0.3354
Epoch [2/5], Step [500/600], Loss: 0.3913
Epoch [2/5], Step [600/600], Loss: 0.3710
Epoch [3/5], Step [100/600], Loss: 0.3028
Epoch [3/5], Step [200/600], Loss: 0.4104
Epoch [3/5], Step [300/600], Loss: 0.2798
Epoch [3/5], Step [400/600], Loss: 0.3226
Epoch [3/5], Step [500/600], Loss: 0.2902
Epoch [3/5], Step [600/600], Loss: 0.3051
Epoch [4/5], Step [100/600], Loss: 0.2763
Epoch [4/5], Step [200/600], Loss: 0.3173
Epoch [4/5], Step [300/600], Loss: 0.3328
Epoch [4/5], Step [400/600], Loss: 0.4096
Epoch [4/5], Step [500/600], Loss: 0.4300
Epoch [4/5]

In [ ]:
run = run_model_one_hidden_size(700, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5986
Epoch [1/5], Step [200/600], Loss: 0.4281
Epoch [1/5], Step [300/600], Loss: 0.4199
Epoch [1/5], Step [400/600], Loss: 0.3539
Epoch [1/5], Step [500/600], Loss: 0.3251
Epoch [1/5], Step [600/600], Loss: 0.4041
Epoch [2/5], Step [100/600], Loss: 0.4327
Epoch [2/5], Step [200/600], Loss: 0.4886
Epoch [2/5], Step [300/600], Loss: 0.2622
Epoch [2/5], Step [400/600], Loss: 0.3085
Epoch [2/5], Step [500/600], Loss: 0.4075
Epoch [2/5], Step [600/600], Loss: 0.3854
Epoch [3/5], Step [100/600], Loss: 0.3643
Epoch [3/5], Step [200/600], Loss: 0.4091
Epoch [3/5], Step [300/600], Loss: 0.2878
Epoch [3/5], Step [400/600], Loss: 0.2426
Epoch [3/5], Step [500/600], Loss: 0.1418
Epoch [3/5], Step [600/600], Loss: 0.3329
Epoch [4/5], Step [100/600], Loss: 0.3865
Epoch [4/5], Step [200/600], Loss: 0.3023
Epoch [4/5], Step [300/600], Loss: 0.2888
Epoch [4/5], Step [400/600], Loss: 0.2322
Epoch [4/5], Step [500/600], Loss: 0.2930
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(100, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 1.3583
Epoch [1/5], Step [200/600], Loss: 1.0657
Epoch [1/5], Step [300/600], Loss: 0.9372
Epoch [1/5], Step [400/600], Loss: 0.8259
Epoch [1/5], Step [500/600], Loss: 0.6676
Epoch [1/5], Step [600/600], Loss: 0.7661
Epoch [2/5], Step [100/600], Loss: 0.6462
Epoch [2/5], Step [200/600], Loss: 0.7662
Epoch [2/5], Step [300/600], Loss: 0.8950
Epoch [2/5], Step [400/600], Loss: 0.7650
Epoch [2/5], Step [500/600], Loss: 0.7962
Epoch [2/5], Step [600/600], Loss: 0.7862
Epoch [3/5], Step [100/600], Loss: 0.6970
Epoch [3/5], Step [200/600], Loss: 0.5935
Epoch [3/5], Step [300/600], Loss: 0.7829
Epoch [3/5], Step [400/600], Loss: 0.7142
Epoch [3/5], Step [500/600], Loss: 0.6215
Epoch [3/5], Step [600/600], Loss: 0.5991
Epoch [4/5], Step [100/600], Loss: 0.5566
Epoch [4/5], Step [200/600], Loss: 0.4926
Epoch [4/5], Step [300/600], Loss: 0.5790
Epoch [4/5], Step [400/600], Loss: 0.7076
Epoch [4/5], Step [500/600], Loss: 0.6533
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(500, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 1.0216
Epoch [1/5], Step [200/600], Loss: 0.9570
Epoch [1/5], Step [300/600], Loss: 0.6637
Epoch [1/5], Step [400/600], Loss: 0.5643
Epoch [1/5], Step [500/600], Loss: 0.6814
Epoch [1/5], Step [600/600], Loss: 0.6344
Epoch [2/5], Step [100/600], Loss: 0.7175
Epoch [2/5], Step [200/600], Loss: 0.7135
Epoch [2/5], Step [300/600], Loss: 0.6280
Epoch [2/5], Step [400/600], Loss: 0.6415
Epoch [2/5], Step [500/600], Loss: 0.7608
Epoch [2/5], Step [600/600], Loss: 0.6743
Epoch [3/5], Step [100/600], Loss: 0.7103
Epoch [3/5], Step [200/600], Loss: 0.7157
Epoch [3/5], Step [300/600], Loss: 0.6635
Epoch [3/5], Step [400/600], Loss: 0.6635
Epoch [3/5], Step [500/600], Loss: 0.6081
Epoch [3/5], Step [600/600], Loss: 0.6550
Epoch [4/5], Step [100/600], Loss: 0.6188
Epoch [4/5], Step [200/600], Loss: 0.6578
Epoch [4/5], Step [300/600], Loss: 0.6454
Epoch [4/5], Step [400/600], Loss: 0.4457
Epoch [4/5], Step [500/600], Loss: 0.6540
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(700, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.9947
Epoch [1/5], Step [200/600], Loss: 0.6729
Epoch [1/5], Step [300/600], Loss: 0.7658
Epoch [1/5], Step [400/600], Loss: 0.7699
Epoch [1/5], Step [500/600], Loss: 0.6877
Epoch [1/5], Step [600/600], Loss: 0.7994
Epoch [2/5], Step [100/600], Loss: 0.5651
Epoch [2/5], Step [200/600], Loss: 0.7284
Epoch [2/5], Step [300/600], Loss: 0.6370
Epoch [2/5], Step [400/600], Loss: 0.5884
Epoch [2/5], Step [500/600], Loss: 0.5155
Epoch [2/5], Step [600/600], Loss: 0.5781
Epoch [3/5], Step [100/600], Loss: 0.5811
Epoch [3/5], Step [200/600], Loss: 0.7031
Epoch [3/5], Step [300/600], Loss: 0.7139
Epoch [3/5], Step [400/600], Loss: 0.5975
Epoch [3/5], Step [500/600], Loss: 0.4548
Epoch [3/5], Step [600/600], Loss: 0.5506
Epoch [4/5], Step [100/600], Loss: 0.6094
Epoch [4/5], Step [200/600], Loss: 0.6833
Epoch [4/5], Step [300/600], Loss: 0.5500
Epoch [4/5], Step [400/600], Loss: 0.5348
Epoch [4/5], Step [500/600], Loss: 0.6134
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(100, 100, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5673
Epoch [1/5], Step [200/600], Loss: 0.6459
Epoch [1/5], Step [300/600], Loss: 0.7025
Epoch [1/5], Step [400/600], Loss: 0.4723
Epoch [1/5], Step [500/600], Loss: 0.4604
Epoch [1/5], Step [600/600], Loss: 0.3975
Epoch [2/5], Step [100/600], Loss: 0.5813
Epoch [2/5], Step [200/600], Loss: 0.3955
Epoch [2/5], Step [300/600], Loss: 0.3770
Epoch [2/5], Step [400/600], Loss: 0.3955
Epoch [2/5], Step [500/600], Loss: 0.4073
Epoch [2/5], Step [600/600], Loss: 0.4419
Epoch [3/5], Step [100/600], Loss: 0.4069
Epoch [3/5], Step [200/600], Loss: 0.3514
Epoch [3/5], Step [300/600], Loss: 0.4735
Epoch [3/5], Step [400/600], Loss: 0.3282
Epoch [3/5], Step [500/600], Loss: 0.3564
Epoch [3/5], Step [600/600], Loss: 0.3713
Epoch [4/5], Step [100/600], Loss: 0.3086
Epoch [4/5], Step [200/600], Loss: 0.3055
Epoch [4/5], Step [300/600], Loss: 0.3837
Epoch [4/5], Step [400/600], Loss: 0.2606
Epoch [4/5], Step [500/600], Loss: 0.3717
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(100, 500, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.7943
Epoch [1/5], Step [200/600], Loss: 0.5722
Epoch [1/5], Step [300/600], Loss: 0.4545
Epoch [1/5], Step [400/600], Loss: 0.5187
Epoch [1/5], Step [500/600], Loss: 0.3510
Epoch [1/5], Step [600/600], Loss: 0.2682
Epoch [2/5], Step [100/600], Loss: 0.3583
Epoch [2/5], Step [200/600], Loss: 0.3307
Epoch [2/5], Step [300/600], Loss: 0.3653
Epoch [2/5], Step [400/600], Loss: 0.3631
Epoch [2/5], Step [500/600], Loss: 0.4665
Epoch [2/5], Step [600/600], Loss: 0.3258
Epoch [3/5], Step [100/600], Loss: 0.3067
Epoch [3/5], Step [200/600], Loss: 0.4817
Epoch [3/5], Step [300/600], Loss: 0.3170
Epoch [3/5], Step [400/600], Loss: 0.3423
Epoch [3/5], Step [500/600], Loss: 0.3542
Epoch [3/5], Step [600/600], Loss: 0.3387
Epoch [4/5], Step [100/600], Loss: 0.4385
Epoch [4/5], Step [200/600], Loss: 0.2239
Epoch [4/5], Step [300/600], Loss: 0.2270
Epoch [4/5], Step [400/600], Loss: 0.3098
Epoch [4/5], Step [500/600], Loss: 0.3435
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(100, 700, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5398
Epoch [1/5], Step [200/600], Loss: 0.6134
Epoch [1/5], Step [300/600], Loss: 0.4168
Epoch [1/5], Step [400/600], Loss: 0.2995
Epoch [1/5], Step [500/600], Loss: 0.4726
Epoch [1/5], Step [600/600], Loss: 0.4499
Epoch [2/5], Step [100/600], Loss: 0.2562
Epoch [2/5], Step [200/600], Loss: 0.2501
Epoch [2/5], Step [300/600], Loss: 0.3760
Epoch [2/5], Step [400/600], Loss: 0.2982
Epoch [2/5], Step [500/600], Loss: 0.5878
Epoch [2/5], Step [600/600], Loss: 0.3267
Epoch [3/5], Step [100/600], Loss: 0.3380
Epoch [3/5], Step [200/600], Loss: 0.2555
Epoch [3/5], Step [300/600], Loss: 0.2826
Epoch [3/5], Step [400/600], Loss: 0.2783
Epoch [3/5], Step [500/600], Loss: 0.2716
Epoch [3/5], Step [600/600], Loss: 0.2377
Epoch [4/5], Step [100/600], Loss: 0.3428
Epoch [4/5], Step [200/600], Loss: 0.3532
Epoch [4/5], Step [300/600], Loss: 0.2151
Epoch [4/5], Step [400/600], Loss: 0.2136
Epoch [4/5], Step [500/600], Loss: 0.2250
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(500, 100, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5063
Epoch [1/5], Step [200/600], Loss: 0.5741
Epoch [1/5], Step [300/600], Loss: 0.4269
Epoch [1/5], Step [400/600], Loss: 0.4146
Epoch [1/5], Step [500/600], Loss: 0.3944
Epoch [1/5], Step [600/600], Loss: 0.3618
Epoch [2/5], Step [100/600], Loss: 0.3522
Epoch [2/5], Step [200/600], Loss: 0.3867
Epoch [2/5], Step [300/600], Loss: 0.2622
Epoch [2/5], Step [400/600], Loss: 0.4232
Epoch [2/5], Step [500/600], Loss: 0.2550
Epoch [2/5], Step [600/600], Loss: 0.3415
Epoch [3/5], Step [100/600], Loss: 0.4248
Epoch [3/5], Step [200/600], Loss: 0.2967
Epoch [3/5], Step [300/600], Loss: 0.2646
Epoch [3/5], Step [400/600], Loss: 0.3191
Epoch [3/5], Step [500/600], Loss: 0.2298
Epoch [3/5], Step [600/600], Loss: 0.3294
Epoch [4/5], Step [100/600], Loss: 0.2464
Epoch [4/5], Step [200/600], Loss: 0.3086
Epoch [4/5], Step [300/600], Loss: 0.2795
Epoch [4/5], Step [400/600], Loss: 0.3025
Epoch [4/5], Step [500/600], Loss: 0.2452
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(500, 500, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.6938
Epoch [1/5], Step [200/600], Loss: 0.5318
Epoch [1/5], Step [300/600], Loss: 0.5932
Epoch [1/5], Step [400/600], Loss: 0.5006
Epoch [1/5], Step [500/600], Loss: 0.3390
Epoch [1/5], Step [600/600], Loss: 0.2914
Epoch [2/5], Step [100/600], Loss: 0.4274
Epoch [2/5], Step [200/600], Loss: 0.4811
Epoch [2/5], Step [300/600], Loss: 0.3621
Epoch [2/5], Step [400/600], Loss: 0.2946
Epoch [2/5], Step [500/600], Loss: 0.3938
Epoch [2/5], Step [600/600], Loss: 0.3453
Epoch [3/5], Step [100/600], Loss: 0.2465
Epoch [3/5], Step [200/600], Loss: 0.2521
Epoch [3/5], Step [300/600], Loss: 0.2600
Epoch [3/5], Step [400/600], Loss: 0.3316
Epoch [3/5], Step [500/600], Loss: 0.2747
Epoch [3/5], Step [600/600], Loss: 0.3710
Epoch [4/5], Step [100/600], Loss: 0.4492
Epoch [4/5], Step [200/600], Loss: 0.3370
Epoch [4/5], Step [300/600], Loss: 0.2952
Epoch [4/5], Step [400/600], Loss: 0.3204
Epoch [4/5], Step [500/600], Loss: 0.2481
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(500, 700, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5369
Epoch [1/5], Step [200/600], Loss: 0.4701
Epoch [1/5], Step [300/600], Loss: 0.2614
Epoch [1/5], Step [400/600], Loss: 0.3426
Epoch [1/5], Step [500/600], Loss: 0.3031
Epoch [1/5], Step [600/600], Loss: 0.4510
Epoch [2/5], Step [100/600], Loss: 0.3007
Epoch [2/5], Step [200/600], Loss: 0.3125
Epoch [2/5], Step [300/600], Loss: 0.3487
Epoch [2/5], Step [400/600], Loss: 0.3830
Epoch [2/5], Step [500/600], Loss: 0.2584
Epoch [2/5], Step [600/600], Loss: 0.2914
Epoch [3/5], Step [100/600], Loss: 0.3529
Epoch [3/5], Step [200/600], Loss: 0.4123
Epoch [3/5], Step [300/600], Loss: 0.2880
Epoch [3/5], Step [400/600], Loss: 0.2404
Epoch [3/5], Step [500/600], Loss: 0.3701
Epoch [3/5], Step [600/600], Loss: 0.3455
Epoch [4/5], Step [100/600], Loss: 0.2430
Epoch [4/5], Step [200/600], Loss: 0.2133
Epoch [4/5], Step [300/600], Loss: 0.2502
Epoch [4/5], Step [400/600], Loss: 0.3656
Epoch [4/5], Step [500/600], Loss: 0.2744
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(700, 100, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.6119
Epoch [1/5], Step [200/600], Loss: 0.5336
Epoch [1/5], Step [300/600], Loss: 0.5977
Epoch [1/5], Step [400/600], Loss: 0.3864
Epoch [1/5], Step [500/600], Loss: 0.5212
Epoch [1/5], Step [600/600], Loss: 0.5762
Epoch [2/5], Step [100/600], Loss: 0.2627
Epoch [2/5], Step [200/600], Loss: 0.4679
Epoch [2/5], Step [300/600], Loss: 0.3743
Epoch [2/5], Step [400/600], Loss: 0.4579
Epoch [2/5], Step [500/600], Loss: 0.2584
Epoch [2/5], Step [600/600], Loss: 0.2595
Epoch [3/5], Step [100/600], Loss: 0.2997
Epoch [3/5], Step [200/600], Loss: 0.3723
Epoch [3/5], Step [300/600], Loss: 0.3434
Epoch [3/5], Step [400/600], Loss: 0.2710
Epoch [3/5], Step [500/600], Loss: 0.2303
Epoch [3/5], Step [600/600], Loss: 0.3046
Epoch [4/5], Step [100/600], Loss: 0.3243
Epoch [4/5], Step [200/600], Loss: 0.1886
Epoch [4/5], Step [300/600], Loss: 0.2415
Epoch [4/5], Step [400/600], Loss: 0.3497
Epoch [4/5], Step [500/600], Loss: 0.3427
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(700, 500, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.4396
Epoch [1/5], Step [200/600], Loss: 0.5612
Epoch [1/5], Step [300/600], Loss: 0.3782
Epoch [1/5], Step [400/600], Loss: 0.4537
Epoch [1/5], Step [500/600], Loss: 0.3985
Epoch [1/5], Step [600/600], Loss: 0.5054
Epoch [2/5], Step [100/600], Loss: 0.6651
Epoch [2/5], Step [200/600], Loss: 0.3704
Epoch [2/5], Step [300/600], Loss: 0.5100
Epoch [2/5], Step [400/600], Loss: 0.3467
Epoch [2/5], Step [500/600], Loss: 0.3879
Epoch [2/5], Step [600/600], Loss: 0.4633
Epoch [3/5], Step [100/600], Loss: 0.2806
Epoch [3/5], Step [200/600], Loss: 0.3632
Epoch [3/5], Step [300/600], Loss: 0.4136
Epoch [3/5], Step [400/600], Loss: 0.2191
Epoch [3/5], Step [500/600], Loss: 0.4411
Epoch [3/5], Step [600/600], Loss: 0.4151
Epoch [4/5], Step [100/600], Loss: 0.2523
Epoch [4/5], Step [200/600], Loss: 0.4112
Epoch [4/5], Step [300/600], Loss: 0.3399
Epoch [4/5], Step [400/600], Loss: 0.4219
Epoch [4/5], Step [500/600], Loss: 0.3769
Epoch [4/5]

In [ ]:
run = run_model_two_hidden_size(700, 700, fashion_mnist_train_data, fashion_mnist_test_data)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.5262
Epoch [1/5], Step [200/600], Loss: 0.5153
Epoch [1/5], Step [300/600], Loss: 0.4201
Epoch [1/5], Step [400/600], Loss: 0.4398
Epoch [1/5], Step [500/600], Loss: 0.3231
Epoch [1/5], Step [600/600], Loss: 0.3607
Epoch [2/5], Step [100/600], Loss: 0.3213
Epoch [2/5], Step [200/600], Loss: 0.3052
Epoch [2/5], Step [300/600], Loss: 0.4506
Epoch [2/5], Step [400/600], Loss: 0.3344
Epoch [2/5], Step [500/600], Loss: 0.2749
Epoch [2/5], Step [600/600], Loss: 0.3646
Epoch [3/5], Step [100/600], Loss: 0.3659
Epoch [3/5], Step [200/600], Loss: 0.2933
Epoch [3/5], Step [300/600], Loss: 0.2663
Epoch [3/5], Step [400/600], Loss: 0.3100
Epoch [3/5], Step [500/600], Loss: 0.3469
Epoch [3/5], Step [600/600], Loss: 0.2423
Epoch [4/5], Step [100/600], Loss: 0.2985
Epoch [4/5], Step [200/600], Loss: 0.2544
Epoch [4/5], Step [300/600], Loss: 0.4229
Epoch [4/5], Step [400/600], Loss: 0.1688
Epoch [4/5], Step [500/600], Loss: 0.2104
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(100, 100, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 1.1386
Epoch [1/5], Step [200/600], Loss: 0.9065
Epoch [1/5], Step [300/600], Loss: 0.8668
Epoch [1/5], Step [400/600], Loss: 0.7358
Epoch [1/5], Step [500/600], Loss: 0.8562
Epoch [1/5], Step [600/600], Loss: 0.9607
Epoch [2/5], Step [100/600], Loss: 0.7292
Epoch [2/5], Step [200/600], Loss: 0.6688
Epoch [2/5], Step [300/600], Loss: 0.9678
Epoch [2/5], Step [400/600], Loss: 0.6575
Epoch [2/5], Step [500/600], Loss: 0.6718
Epoch [2/5], Step [600/600], Loss: 0.7351
Epoch [3/5], Step [100/600], Loss: 0.5921
Epoch [3/5], Step [200/600], Loss: 0.6971
Epoch [3/5], Step [300/600], Loss: 0.7504
Epoch [3/5], Step [400/600], Loss: 0.6787
Epoch [3/5], Step [500/600], Loss: 0.6726
Epoch [3/5], Step [600/600], Loss: 0.6571
Epoch [4/5], Step [100/600], Loss: 0.7372
Epoch [4/5], Step [200/600], Loss: 0.7409
Epoch [4/5], Step [300/600], Loss: 0.5651
Epoch [4/5], Step [400/600], Loss: 0.6706
Epoch [4/5], Step [500/600], Loss: 0.5150
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(100, 500, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 1.1552
Epoch [1/5], Step [200/600], Loss: 0.6703
Epoch [1/5], Step [300/600], Loss: 0.9218
Epoch [1/5], Step [400/600], Loss: 0.7001
Epoch [1/5], Step [500/600], Loss: 0.6739
Epoch [1/5], Step [600/600], Loss: 0.6864
Epoch [2/5], Step [100/600], Loss: 0.6712
Epoch [2/5], Step [200/600], Loss: 0.6094
Epoch [2/5], Step [300/600], Loss: 0.6935
Epoch [2/5], Step [400/600], Loss: 0.4914
Epoch [2/5], Step [500/600], Loss: 0.6189
Epoch [2/5], Step [600/600], Loss: 0.5408
Epoch [3/5], Step [100/600], Loss: 0.7563
Epoch [3/5], Step [200/600], Loss: 0.5482
Epoch [3/5], Step [300/600], Loss: 0.6462
Epoch [3/5], Step [400/600], Loss: 0.5997
Epoch [3/5], Step [500/600], Loss: 0.4312
Epoch [3/5], Step [600/600], Loss: 0.5974
Epoch [4/5], Step [100/600], Loss: 0.5825
Epoch [4/5], Step [200/600], Loss: 0.6710
Epoch [4/5], Step [300/600], Loss: 0.5942
Epoch [4/5], Step [400/600], Loss: 0.6283
Epoch [4/5], Step [500/600], Loss: 0.4490
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(100, 700, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.9229
Epoch [1/5], Step [200/600], Loss: 0.7435
Epoch [1/5], Step [300/600], Loss: 0.7489
Epoch [1/5], Step [400/600], Loss: 0.7692
Epoch [1/5], Step [500/600], Loss: 0.7075
Epoch [1/5], Step [600/600], Loss: 0.5527
Epoch [2/5], Step [100/600], Loss: 0.7181
Epoch [2/5], Step [200/600], Loss: 0.6311
Epoch [2/5], Step [300/600], Loss: 0.5267
Epoch [2/5], Step [400/600], Loss: 0.7100
Epoch [2/5], Step [500/600], Loss: 0.4505
Epoch [2/5], Step [600/600], Loss: 0.6127
Epoch [3/5], Step [100/600], Loss: 0.5230
Epoch [3/5], Step [200/600], Loss: 0.6224
Epoch [3/5], Step [300/600], Loss: 0.6146
Epoch [3/5], Step [400/600], Loss: 0.5277
Epoch [3/5], Step [500/600], Loss: 0.3951
Epoch [3/5], Step [600/600], Loss: 0.4110
Epoch [4/5], Step [100/600], Loss: 0.7721
Epoch [4/5], Step [200/600], Loss: 0.5936
Epoch [4/5], Step [300/600], Loss: 0.7232
Epoch [4/5], Step [400/600], Loss: 0.5426
Epoch [4/5], Step [500/600], Loss: 0.3887
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(500, 100, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.9331
Epoch [1/5], Step [200/600], Loss: 0.8915
Epoch [1/5], Step [300/600], Loss: 0.7870
Epoch [1/5], Step [400/600], Loss: 0.4414
Epoch [1/5], Step [500/600], Loss: 0.7120
Epoch [1/5], Step [600/600], Loss: 0.6934
Epoch [2/5], Step [100/600], Loss: 0.6513
Epoch [2/5], Step [200/600], Loss: 0.5642
Epoch [2/5], Step [300/600], Loss: 0.6640
Epoch [2/5], Step [400/600], Loss: 0.6009
Epoch [2/5], Step [500/600], Loss: 0.6057
Epoch [2/5], Step [600/600], Loss: 0.5411
Epoch [3/5], Step [100/600], Loss: 0.5823
Epoch [3/5], Step [200/600], Loss: 0.5926
Epoch [3/5], Step [300/600], Loss: 0.6007
Epoch [3/5], Step [400/600], Loss: 0.6623
Epoch [3/5], Step [500/600], Loss: 0.5392
Epoch [3/5], Step [600/600], Loss: 0.7429
Epoch [4/5], Step [100/600], Loss: 0.6958
Epoch [4/5], Step [200/600], Loss: 0.5844
Epoch [4/5], Step [300/600], Loss: 0.6032
Epoch [4/5], Step [400/600], Loss: 0.5714
Epoch [4/5], Step [500/600], Loss: 0.6580
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(500, 500, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.8425
Epoch [1/5], Step [200/600], Loss: 0.8045
Epoch [1/5], Step [300/600], Loss: 0.7395
Epoch [1/5], Step [400/600], Loss: 0.6405
Epoch [1/5], Step [500/600], Loss: 0.5755
Epoch [1/5], Step [600/600], Loss: 0.6018
Epoch [2/5], Step [100/600], Loss: 0.5480
Epoch [2/5], Step [200/600], Loss: 0.5836
Epoch [2/5], Step [300/600], Loss: 0.6867
Epoch [2/5], Step [400/600], Loss: 0.5570
Epoch [2/5], Step [500/600], Loss: 0.4792
Epoch [2/5], Step [600/600], Loss: 0.5716
Epoch [3/5], Step [100/600], Loss: 0.4276
Epoch [3/5], Step [200/600], Loss: 0.3960
Epoch [3/5], Step [300/600], Loss: 0.5113
Epoch [3/5], Step [400/600], Loss: 0.4834
Epoch [3/5], Step [500/600], Loss: 0.6397
Epoch [3/5], Step [600/600], Loss: 0.5682
Epoch [4/5], Step [100/600], Loss: 0.4524
Epoch [4/5], Step [200/600], Loss: 0.4826
Epoch [4/5], Step [300/600], Loss: 0.5325
Epoch [4/5], Step [400/600], Loss: 0.4461
Epoch [4/5], Step [500/600], Loss: 0.6761
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(500, 700, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.8690
Epoch [1/5], Step [200/600], Loss: 0.6108
Epoch [1/5], Step [300/600], Loss: 0.5300
Epoch [1/5], Step [400/600], Loss: 0.7885
Epoch [1/5], Step [500/600], Loss: 0.5017
Epoch [1/5], Step [600/600], Loss: 0.5933
Epoch [2/5], Step [100/600], Loss: 0.6679
Epoch [2/5], Step [200/600], Loss: 0.7227
Epoch [2/5], Step [300/600], Loss: 0.7476
Epoch [2/5], Step [400/600], Loss: 0.5000
Epoch [2/5], Step [500/600], Loss: 0.5118
Epoch [2/5], Step [600/600], Loss: 0.4888
Epoch [3/5], Step [100/600], Loss: 0.5832
Epoch [3/5], Step [200/600], Loss: 0.6142
Epoch [3/5], Step [300/600], Loss: 0.3288
Epoch [3/5], Step [400/600], Loss: 0.6777
Epoch [3/5], Step [500/600], Loss: 0.4932
Epoch [3/5], Step [600/600], Loss: 0.5388
Epoch [4/5], Step [100/600], Loss: 0.5368
Epoch [4/5], Step [200/600], Loss: 0.5569
Epoch [4/5], Step [300/600], Loss: 0.4895
Epoch [4/5], Step [400/600], Loss: 0.5386
Epoch [4/5], Step [500/600], Loss: 0.5570
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(700, 100, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.8667
Epoch [1/5], Step [200/600], Loss: 0.7164
Epoch [1/5], Step [300/600], Loss: 0.6273
Epoch [1/5], Step [400/600], Loss: 0.7224
Epoch [1/5], Step [500/600], Loss: 1.0187
Epoch [1/5], Step [600/600], Loss: 0.5848
Epoch [2/5], Step [100/600], Loss: 0.7186
Epoch [2/5], Step [200/600], Loss: 0.5853
Epoch [2/5], Step [300/600], Loss: 0.7229
Epoch [2/5], Step [400/600], Loss: 0.4827
Epoch [2/5], Step [500/600], Loss: 0.4882
Epoch [2/5], Step [600/600], Loss: 0.4837
Epoch [3/5], Step [100/600], Loss: 0.6650
Epoch [3/5], Step [200/600], Loss: 0.5831
Epoch [3/5], Step [300/600], Loss: 0.5354
Epoch [3/5], Step [400/600], Loss: 0.7125
Epoch [3/5], Step [500/600], Loss: 0.6250
Epoch [3/5], Step [600/600], Loss: 0.4817
Epoch [4/5], Step [100/600], Loss: 0.4826
Epoch [4/5], Step [200/600], Loss: 0.5024
Epoch [4/5], Step [300/600], Loss: 0.4909
Epoch [4/5], Step [400/600], Loss: 0.6297
Epoch [4/5], Step [500/600], Loss: 0.4347
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(700, 500, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.8048
Epoch [1/5], Step [200/600], Loss: 0.6949
Epoch [1/5], Step [300/600], Loss: 0.7533
Epoch [1/5], Step [400/600], Loss: 0.7716
Epoch [1/5], Step [500/600], Loss: 0.7043
Epoch [1/5], Step [600/600], Loss: 0.7007
Epoch [2/5], Step [100/600], Loss: 0.6977
Epoch [2/5], Step [200/600], Loss: 0.5395
Epoch [2/5], Step [300/600], Loss: 0.5797
Epoch [2/5], Step [400/600], Loss: 0.5878
Epoch [2/5], Step [500/600], Loss: 0.4764
Epoch [2/5], Step [600/600], Loss: 0.5538
Epoch [3/5], Step [100/600], Loss: 0.4543
Epoch [3/5], Step [200/600], Loss: 0.5100
Epoch [3/5], Step [300/600], Loss: 0.6335
Epoch [3/5], Step [400/600], Loss: 0.4194
Epoch [3/5], Step [500/600], Loss: 0.5051
Epoch [3/5], Step [600/600], Loss: 0.6505
Epoch [4/5], Step [100/600], Loss: 0.4765
Epoch [4/5], Step [200/600], Loss: 0.4604
Epoch [4/5], Step [300/600], Loss: 0.4268
Epoch [4/5], Step [400/600], Loss: 0.5522
Epoch [4/5], Step [500/600], Loss: 0.5352
Epoch [4/5]

In [ ]:
run = run_model_feature_two_hidden_size(700, 700, fashion_mnist_train_data, fashion_mnist_test_data, True)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 0.9081
Epoch [1/5], Step [200/600], Loss: 0.8301
Epoch [1/5], Step [300/600], Loss: 0.7582
Epoch [1/5], Step [400/600], Loss: 0.5810
Epoch [1/5], Step [500/600], Loss: 0.5200
Epoch [1/5], Step [600/600], Loss: 0.5337
Epoch [2/5], Step [100/600], Loss: 0.6206
Epoch [2/5], Step [200/600], Loss: 0.5225
Epoch [2/5], Step [300/600], Loss: 0.5765
Epoch [2/5], Step [400/600], Loss: 0.5948
Epoch [2/5], Step [500/600], Loss: 0.4712
Epoch [2/5], Step [600/600], Loss: 0.6161
Epoch [3/5], Step [100/600], Loss: 0.7375
Epoch [3/5], Step [200/600], Loss: 0.5464
Epoch [3/5], Step [300/600], Loss: 0.5589
Epoch [3/5], Step [400/600], Loss: 0.5311
Epoch [3/5], Step [500/600], Loss: 0.6740
Epoch [3/5], Step [600/600], Loss: 0.6716
Epoch [4/5], Step [100/600], Loss: 0.4511
Epoch [4/5], Step [200/600], Loss: 0.6202
Epoch [4/5], Step [300/600], Loss: 0.5092
Epoch [4/5], Step [400/600], Loss: 0.7085
Epoch [4/5], Step [500/600], Loss: 0.7154
Epoch [4/5]

In [ ]:
run = run_model_feature_one_hidden_size(700, fashion_mnist_train_data, fashion_mnist_test_data, False)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 1.9366
Epoch [1/5], Step [200/600], Loss: 1.8149
Epoch [1/5], Step [300/600], Loss: 1.6375
Epoch [1/5], Step [400/600], Loss: 1.6728
Epoch [1/5], Step [500/600], Loss: 1.4309
Epoch [1/5], Step [600/600], Loss: 1.5043
Epoch [2/5], Step [100/600], Loss: 1.5261
Epoch [2/5], Step [200/600], Loss: 1.5602
Epoch [2/5], Step [300/600], Loss: 1.4849
Epoch [2/5], Step [400/600], Loss: 1.4576
Epoch [2/5], Step [500/600], Loss: 1.4019
Epoch [2/5], Step [600/600], Loss: 1.3089
Epoch [3/5], Step [100/600], Loss: 1.4515
Epoch [3/5], Step [200/600], Loss: 1.4860
Epoch [3/5], Step [300/600], Loss: 1.4182
Epoch [3/5], Step [400/600], Loss: 1.4279
Epoch [3/5], Step [500/600], Loss: 1.3960
Epoch [3/5], Step [600/600], Loss: 1.4664
Epoch [4/5], Step [100/600], Loss: 1.4289
Epoch [4/5], Step [200/600], Loss: 1.3946
Epoch [4/5], Step [300/600], Loss: 1.3792
Epoch [4/5], Step [400/600], Loss: 1.3025
Epoch [4/5], Step [500/600], Loss: 1.4929
Epoch [4/5]

In [10]:
run = run_model_feature_two_hidden_size(700, 500, fashion_mnist_train_data, fashion_mnist_test_data, False)

Training on dataset...
Epoch [1/5], Step [100/600], Loss: 1.8023
Epoch [1/5], Step [200/600], Loss: 1.6402
Epoch [1/5], Step [300/600], Loss: 1.7849
Epoch [1/5], Step [400/600], Loss: 1.4629
Epoch [1/5], Step [500/600], Loss: 1.6073
Epoch [1/5], Step [600/600], Loss: 1.4274
Epoch [2/5], Step [100/600], Loss: 1.4493
Epoch [2/5], Step [200/600], Loss: 1.5048
Epoch [2/5], Step [300/600], Loss: 1.3485
Epoch [2/5], Step [400/600], Loss: 1.3511
Epoch [2/5], Step [500/600], Loss: 1.4362
Epoch [2/5], Step [600/600], Loss: 1.2342
Epoch [3/5], Step [100/600], Loss: 1.1857
Epoch [3/5], Step [200/600], Loss: 1.3532
Epoch [3/5], Step [300/600], Loss: 1.4636
Epoch [3/5], Step [400/600], Loss: 1.3756
Epoch [3/5], Step [500/600], Loss: 1.4899
Epoch [3/5], Step [600/600], Loss: 1.2601
Epoch [4/5], Step [100/600], Loss: 1.3493
Epoch [4/5], Step [200/600], Loss: 1.0235
Epoch [4/5], Step [300/600], Loss: 1.1120
Epoch [4/5], Step [400/600], Loss: 1.2592
Epoch [4/5], Step [500/600], Loss: 1.3288
Epoch [4/5]

Nhận xét:  
- Khi tăng số lớp ẩn lên thì khi chọn tổ hợp lớp ẩn tốt nhất thì nó có cải thiện độ chính xác của mô hình, dù là so sánh với dùng một lớp ẩn có 700 nơ-ron, nhìn chung thì độ chính xác sẽ có tăng, đồng thời gian train cũng có tăng.
- Khi chúng ta rút đặc trưng thì độ chính xác ở 2 lớp ẩn cũng cao hơn so với một lớp ẩn nhưng khi so kết quả tốt nhất của 2 lớp ẩn có rút đặc trưng với 2 lớp ẩn không rút đặc trưng thì hai lớp ẩn không rút đặc trưng vẫn tốt hơn một ít những không quá lớn cụ thể là 97.94 ở không rút đặc trưng với tổ hợp (500,500) và có rút vẫn (500,500) là 97,62. Fashion mnist cũng tương tự không rút đặc trưng ở (700,100) là 88,29, còn có rút ở tổ hợp (500,700) là 80,74. Nên việc rút đặc trưng lợi thể vẫn là tốc độ train nhanh, mặc dù ở mnist thì không nhiều nhưng ở Fashion mnist thì độ chính xác có hơi lệch nhiều.
- Hai lớp ẩn thì khi ta dùng Histogram nó vẫn cho kết quả không khả quan hơn là bao.
- Tóm lại dựa vào các kết quả chạy ở trên việc sử dụng mạng nơ-ron sâu sẽ cho kết quả tốt hơn mạng nơ-ron nông.
- Rút trích dặc trưng dù kết hợp với mạng nơ-ron sâu vẫn không thể đánh bại việc dùng trực tiếp với mạng nơ-ron sâu nên việc rút trích đặc trưng lợi thế lớn nhất của nó vẫn là thời gian train.